In [17]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubun

In [18]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-27 21:54:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-09-27 21:54:19 (10.6 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [20]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
cameras_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True)
cameras_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [21]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watches_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True)
watches_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

Cameras Vine Reviews:

In [22]:
from pyspark import SparkFiles
vine_table_cameras = cameras_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_cameras.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1NBG94582SJE2|          5|            0|          0|   N|
|R273DCA6Y0H9V7|          5|            0|          0|   N|
| RQVOXO7WUOFK6|          2|            1|          1|   N|
|R1KWKSF21PO6HO|          5|            0|          0|   N|
|R38H3UO1J190GI|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



Watches Vine Review:

In [23]:
from pyspark import SparkFiles
vine_table_watches = watches_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_watches.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3O9SGZBVQBV76|          5|            0|          0|   N|
| RKH8BNC3L5DLF|          5|            0|          0|   N|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|
|R31U3UH5AZ42LL|          5|            0|          0|   N|
|R2SV659OUJ945Y|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



Drop NA and Drop Duplicates

In [24]:
print(vine_table_cameras.count())
print(vine_table_watches.count())
vine_table_watches = vine_table_watches.dropna()
vine_table_watches = vine_table_watches.na.drop()
vine_table_watches = vine_table_watches.dropDuplicates()
vine_table_cameras = vine_table_cameras.dropna()
vine_table_cameras = vine_table_cameras.na.drop()
vine_table_cameras = vine_table_cameras.dropDuplicates()
print(vine_table_cameras.count())
print(vine_table_watches.count())

1801974
960872
1801972
960872


In [25]:
vine_table_watches.describe().show()

+-------+--------------+------------------+------------------+------------------+------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|  vine|
+-------+--------------+------------------+------------------+------------------+------+
|  count|        960872|            960872|            960872|            960872|960872|
|   mean|          null| 4.138282726523408|1.1968274650525772|1.5611049130373245|  null|
| stddev|          null|1.2932852524784497|  8.16113672841207|   9.0687961219924|  null|
|    min|R100047PHC0PC5|                 1|                 0|                 0|     N|
|    max| RZZZRO24XW75Z|                 5|                99|                99|     Y|
+-------+--------------+------------------+------------------+------------------+------+



In [26]:
  vine_table_cameras.describe().show()

+-------+--------------+------------------+------------------+------------------+-------+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|   vine|
+-------+--------------+------------------+------------------+------------------+-------+
|  count|       1801972|           1801972|           1801972|           1801972|1801972|
|   mean|          null| 4.127014737187925| 2.905414179576597| 3.638446102381169|   null|
| stddev|          null|1.3053194462552795|23.099167639906334|24.505995368170765|   null|
|    min|R10003UWRY0AVF|                 1|                 0|                 0|      N|
|    max| RZZZUWFYAAIHI|                 5|               999|               999|      Y|
+-------+--------------+------------------+------------------+------------------+-------+



In [42]:
from pyspark.sql.types import IntegerType
from pyspark import SparkFiles
vine_table_watches = vine_table_watches.withColumn("star_rating",watches_df["star_rating"].cast(IntegerType()))
vine_table_watches.groupBy("vine").avg("star_rating").show(truncate=False)

+----+-----------------+
|vine|avg(star_rating) |
+----+-----------------+
|Y   |4.034344590726961|
|N   |4.13847204483253 |
+----+-----------------+



In [44]:
from pyspark.sql.types import IntegerType
from pyspark import SparkFiles
vine_table_cameras = vine_table_cameras.withColumn("star_rating",cameras_df["star_rating"].cast(IntegerType()))
vine_table_cameras.groupBy("vine").avg("star_rating").show(truncate=False)

+----+-----------------+
|vine|avg(star_rating) |
+----+-----------------+
|Y   |4.128250665990105|
|N   |4.127009306673192|
+----+-----------------+



In [48]:
from pyspark import SparkFiles
from pyspark.sql.functions import avg, col, desc
vine_table_watches.groupBy("star_rating", "vine").count().sort(col("star_rating").desc()).show(truncate=False)

+-----------+----+------+
|star_rating|vine|count |
+-----------+----+------+
|5          |Y   |605   |
|5          |N   |571022|
|4          |Y   |754   |
|4          |N   |172125|
|3          |N   |78888 |
|3          |Y   |266   |
|2          |Y   |87    |
|2          |N   |51948 |
|1          |N   |85142 |
|1          |Y   |35    |
+-----------+----+------+

